# L13.1 Naive Bayes Practice
---
이름: 김태산

학과: 자동차공학과

학번: 20143211

---

### 과제 목표: Naive Bayes 정리를 사용하여 SMS 스팸 데이터로부터 스팸 메시지와 일반 메시지를 분류하는 방법을 알아본다.

### 모듈 불러오기
- 사용할 모듈 import 하기

In [2]:
import zipfile
import requests
from collections import Counter
import re
import math

### 데이터 불러오기
- requests.get을 통해 아래의 url에서 SMS 스팸 데이터 압축 파일을 직접 다운로드하여 저장하고, zipfile을 이용하여 압축 파일의 모든 파일을 압축 해제한다.
- list comprehension을 통해 SMS 스팸 데이터를 불러온다.
- data의 첫 10행을 출력해보면, 각각의 SMS마다 스팸 여부와 SMS 내용이 나뉘어 각각의 리스트로 입력되어 있는 것을 알 수 있다.


In [9]:
mail = requests.get("http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/smsspamcollection.zip")

with open("/content/drive/MyDrive/datasets/SMS spam/spam.zip", 'wb') as f:
  f.write(mail.content)

zipfile.ZipFile("/content/drive/MyDrive/datasets/SMS spam/spam.zip").extractall("/content/drive/MyDrive/datasets/SMS spam")

data = [line.strip().split("\t") for line in open("/content/drive/MyDrive/datasets/SMS spam/SMSSpamCollection.txt")]
data[:10]

[['ham',
  'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'],
 ['ham', 'Ok lar... Joking wif u oni...'],
 ['spam',
  "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"],
 ['ham', 'U dun say so early hor... U c already then say...'],
 ['ham', "Nah I don't think he goes to usf, he lives around here though"],
 ['spam',
  "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv"],
 ['ham',
  'Even my brother is not like to speak with me. They treat me like aids patent.'],
 ['ham',
  "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune"],
 ['spam',
  'WINNER!! As a valued network customer you have been selected to receivea £

### 데이터 나누기
- train, test 데이터 분리
  - train 데이터는 전체 데이터의 80%를,  test 데이터는 train 데이터를 제외한 나머지 20%를 사용한다.

In [11]:
train_size = int(len(data) * 0.8)
train = data[:train_size]
test = data[train_size:]

### 사전확률 구하기
- 스팸 메시지와 일반 메시지의 갯수를 세어 사전확률 구하기
 - 먼저 Counter를 이용하여 스팸 메시지와 일반 메시지의 갯수를 센다.
 - 아래의 출력을 보면, 스팸 메시지는 602개, 일반 메시지는 3857개인 것을 알 수 있다.
 - 각각의 메시지 갯수에서 전체 메시지 갯수를 나눠 스팸 메시지와 일반 메시지의 사전확률을 아래와 같이 계산한다.

In [53]:
sms_counter = Counter(s[0] for s in train)
prior_spam = sms_counter['spam'] / (sms_counter['ham'] + sms_counter['spam'])
prior_ham = sms_counter['ham'] / (sms_counter['ham'] + sms_counter['spam'])
print("# of spams:", sms_counter["spam"])
print("# of hams: ", sms_counter["ham"])
print("Prior prob. of spam:", prior_spam)
print("Prior prob. of ham: ", prior_ham)

# of spams: 602
# of hams:  3857
Prior prob. of spam: 0.13500784929356358
Prior prob. of ham:  0.8649921507064364


### Word Count
- 스팸 메시지들에서 각 단어가 등장한 횟수 세기
  - train 데이터에서 spam으로 분류된 메시지의 문장을 정규표현식을 사용하여 소문자 알파벳, 숫자, _의 한 문자 이상의 문자열로 나누어 iterator로 만든 후 이를 Counter에 입력하여 스팸 메시지의 전체 단어와 단어의 등장 횟수를 구한다.
- 일반 메시지들에서 각 단어가 등장한 횟수 세기
  - train 데이터에서 ham으로 분류된 메시지의 문장을 정규표현식을 사용하여 소문자 알파벳, 숫자, _의 한 문자 이상의 문자열로 나누어 iterator로 만든 후 이를 Counter에 입력하여 일반 메시지의 전체 단어와 단어의 등장 횟수를 구한다.

In [44]:
spam_words = Counter(word.group() for s, sent in train if s == "spam" \
                            for word in re.finditer("[0-9a-z_]+", sent.lower()))
ham_words = Counter(word.group() for s, sent in train if s == "ham" \
                         for word in re.finditer("[0-9a-z_]+", sent.lower()))

- 단어 집합 크기 구하기
 - train 데이터에 등장하는 전체 단어를 set을 통해 집합으로 만들면 중복된 단어들이 제외되어 1번 이상 등장한 단어의 갯수를 셀 수 있다.
 - num_unique_words는 뒤에서 라플라스 스무딩을 통해 모든 단어가 기본적으로 k번은 등장한 것으로 처리할 때 사용된다.
- 스팸 메시지의 전체 단어 수 세기
  - 스팸 메시지의 전체 단어 수는 spam_words.values()를 모두 합하여 구한다.
- 일반 메시지의 전체 단어 수 세기
 - 일반 메시지의 전체 단어 수는 ham_words.values()를 모두 합하여 구한다.

In [50]:
num_unique_words = len(set(word.group() for s, sent in train \
                                for word in re.finditer("[a-z0-9_]+", sent.lower())))
num_spam_words = sum(spam_words.values())
num_ham_words = sum(ham_words.values())
print("# of unique words:", num_unique_words)
print("# of words in spams:", num_spam_words)
print("# of words in hams:", num_ham_words)

# of unique words: 7810
# of words in spams: 15341
# of words in hams: 57097


### predict()
- naive bayes 방법에 따라 spam/ham 점수를 구하여 비교
 - naive bayes 정리를 이용하여 메시지의 단어가 등장하였을 때 스팸 또는 일반 메시지일 확률 P(S|w)을 계산한다.
 - P(S|w)는 P(S) * P(w|S) / P(w)의 식으로 구할 수 있는데, P(w)는 스팸과 일반 메시지에서 공통된 부분이므로 P(S) * P(w|S)만 계산하여 비교하면 된다.
 - 스팸 메시지와 일반 메시지에 대해 P(S)는 각각 위에서 구한 prior_spam과 prior_ham이고, P(w|S)는 아래의 코드에서 spam_words[w] / spam_denom과 ham_words[w] / ham_denom 부분이다.
- 라플라스 스무딩 적용
 - num_unique_words에 k를 곱한 것을 더하여 모든 단어가 최소 k번 이상은 등장한 것으로 가정한다.
 - 스무딩을 적용함으로써 한번도 등장하지 않은 단어의 횟수가 0이 되는 것을 방지하여 로그 계산이 가능하도록 한다.
- 로그 합산
 - 메시지에 등장하는 단어의 수가 많아 계속해서 확률값을 곱하게 되면 점수가 0에 가까워져 컴퓨터 연산의 특성상 정확도가 떨어지게 되는데 이를 방지하기 위해 확률값에 로그를 취하여 합산한다.

In [57]:
def predict(txt, k=0.5):
  words = re.findall("[0-9a-z_]+", txt.lower())

  spam_denom = num_spam_words + num_unique_words * k
  ham_denom = num_ham_words + num_unique_words * k
  
  spam_score = math.log(prior_spam)
  ham_score = math.log(prior_ham)

  for w in words:
    spam_score += math.log((spam_words[w] + k) / spam_denom)
    ham_score += math.log((ham_words[w] + k) / ham_denom)
  
  return spam_score > ham_score

### 테스트 해보기
- 잘 맞히는지 확인해보기
 - 아래에서 여러 지표를 비교하기 위해 예측 결과를 이용하여 true positive, true negative, false positive, false negative를 구한다.
 - 메시지가 실제로 스팸 메시지이고 예측 결과도 스팸 메시지인 경우 true positive를 1 증가시킨다.
 - 메시지가 실제로 일반 메시지이고 예측 결과도 일반 메시지인 경우 true negative를 1 증가시킨다.
 - 메시지가 실제로는 일반 메시지이지만 예측 결과는 스팸 메시지인 경우 false positive를 1 증가시킨다.
  - 메시지가 실제로는 스팸 메시지이지만 예측 결과는 일반 메시지인 경우 false negative를 1 증가시킨다.

In [59]:
tp = 0
tn = 0
fp = 0
fn = 0

for s, sent in test:
  pred = predict(sent)
  if s == "spam" and pred:
    tp += 1
  elif s != "spam" and not pred:
    tn += 1
  elif s != "spam" and pred:
    fp += 1
  else:
    fn += 1

- Precision, Recall, F1 score, Accuracy
 - precision은 positive라고 예측한 것 중 실제로 positive인 것의 비율을 의미한다.
 - recall은 positive 전체 중에서 positive라고 예측한 것의 비율을 의미한다.
 - f1 score는 precision과 recall의 조화평균을 의미한다.
 - accuracy는 전체 데이터 중에서 정확히 positive와 negative를 예측한 것의 비율을 의미한다.

In [60]:
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)
accuracy = (tp + tn) / (tp + tn + fp + fn)

print("precision:", precision)
print("recall:", recall)
print("f1 score:", f1)
print("accuracy:", accuracy)

precision: 0.9090909090909091
recall: 0.9655172413793104
f1 score: 0.9364548494983278
accuracy: 0.9829596412556054


- precision을 보면, 스팸 메시지라고 예측할 경우 약 90.9%는 실제로 스팸 메시지인 것을 알 수 있다.
- recall을 보면, 전체 스팸 메시지 중에서 약 96.6%를 스팸 메시지로 분류한다는 것을 알 수 있다.
- precision과 recall의 조화 평균 f1 score는 0.936 정도인 것을 알 수 있다.
- accuracy를 보면, 전체 메시지의 약 98.2%를 스팸 메시지인지 일반 메시지인지 정확하게 분류한다는 것을 알 수 있다.